# 多输入通道和多输出通道
前⾯两节⾥我们⽤到的输⼊和输出都是⼆维数组，但真实数据的维度经常更⾼。例如，彩⾊图像
在⾼和宽2个维度外还有RGB（红、绿、蓝） 3个颜⾊通道。假设彩⾊图像的⾼和宽分别是h和w（像
素），那么它可以表⽰为⼀个$3 × h × w$的多维数组。我们将⼤小为3的这⼀维称为通道（channel）
维。本节我们将介绍含多个输⼊通道或多个输出通道的卷积核
## 多输⼊通道
当输⼊数据含多个通道时，我们需要构造⼀个输⼊通道数与输⼊数据的通道数相同的卷积核，从
而能够与含多通道的输⼊数据做互相关运算。假设输⼊数据的通道数为$ci$，那么卷积核的输⼊
通道数同样为$c_i$。设卷积核窗口形状为$k_h × k_w$。当$c_i = 1$时，我们知道卷积核只包含⼀个形状
为$k_h × k_w$的⼆维数组。当$c_i > 1$时，我们将会为每个输⼊通道各分配⼀个形状为$k_h × k_w$的核数
组。把这$c_i$个数组在输⼊通道维上连结，即得到⼀个形状为$c_i × k_h × k_w$的卷积核。由于输⼊和卷
积核各有$c_i$个通道，我们可以在各个通道上对输⼊的⼆维数组和卷积核的⼆维核数组做互相关运
算，再将这$c_i$个互相关运算的⼆维输出按通道相加，得到⼀个⼆维数组。这就是含多个通道的输
⼊数据与多输⼊通道的卷积核做⼆维互相关运算的输出。

图5.4展⽰了含2个输⼊通道的⼆维互相关计算的例⼦。在每个通道上，⼆维输⼊数组与⼆维核数
组做互相关运算，再按通道相加即得到输出。图5.4中阴影部分为第⼀个输出元素及其计算所使
⽤的输⼊和核数组元素： $(1 × 1 + 2 × 2 + 4 × 3 + 5 × 4) + (0 × 0 + 1 × 1 + 3 × 2 + 4 × 3) = 56。$

![含2个输入通道的互相关计算](../img/conv_multi_in.svg)

接下来我们实现含多个输⼊通道的互相关运算。我们只需要对每个通道做互相关运算，然后通
过add_n函数来进⾏累加。

In [3]:
import d2lzh as d2l
from mxnet import nd
def corr2d_multi_in(X, K):
    # ⾸先沿着X和K的第0维（通道维）遍历。然后使⽤*将结果列表变成add_n函数的位置参数
    # （positional argument）来进⾏相加
    return nd.add_n(*[d2l.corr2d(x,k) for x,k in zip(X,K)])

我们可以构造图5.4中的输⼊数组X、核数组K来验证互相关运算的输出。

In [4]:
X = nd.array([[[0,1,2],[3,4,5],[6,7,8]],
              [[1,2,3],[4,5,6],[7,8,9]]])
K = nd.array([[[0,1],[2,3]],
              [[1,2],[3,4]]])
corr2d_multi_in(X,K)


[[ 56.  72.]
 [104. 120.]]
<NDArray 2x2 @cpu(0)>

## 多输出通道
当输⼊通道有多个时，因为我们对各个通道的结果做了累加，所以不论输⼊通道数是多少，输出
通道数总是为1。设卷积核输⼊通道数和输出通道数分别为$c_i和c_o，⾼和宽分别为k_h和k_w$。如果希
望得到含多个通道的输出，我们可以为每个输出通道分别创建形状为$c_i × k_h × k_w$的核数组。将
它们在输出通道维上连结，卷积核的形状即$c_o × c_i × k_h × k_w$。在做互相关运算时，每个输出通
道上的结果由卷积核在该输出通道上的核数组与整个输⼊数组计算而来。
下⾯我们实现⼀个互相关运算函数来计算多个通道的输出。

In [6]:
def corr2d_multi_in_out(X, K):
    # 对K的第0维遍历，每次同输⼊X做互相关计算。所有结果使⽤stack函数合并在⼀起
    return nd.stack(*[corr2d_multi_in(X,k) for k in K])

我们将核数组K同K+1（K中每个元素加⼀）和K+2连结在⼀起来构造⼀个输出通道数为3的卷积核。

In [9]:
K = nd.stack(K, K + 1, K + 2)
K.shape

(3, 2, 2, 2)

下⾯我们对输⼊数组X与核数组K做互相关运算。此时的输出含有3个通道。其中第⼀个通道的结
果与之前输⼊数组X与多输⼊通道、单输出通道核的计算结果⼀致

In [10]:
corr2d_multi_in_out(X, K)


[[[ 56.  72.]
  [104. 120.]]

 [[ 76. 100.]
  [148. 172.]]

 [[ 96. 128.]
  [192. 224.]]]
<NDArray 3x2x2 @cpu(0)>

## 1 × 1卷积层
最后我们讨论卷积窗口形状为1 × 1（kh = kw = 1）的多通道卷积层。我们通常称之为1 × 1卷
积层，并将其中的卷积运算称为1 × 1卷积。因为使⽤了最小窗口， 1 × 1卷积失去了卷积层可以
识别⾼和宽维度上相邻元素构成的模式的功能。实际上， 1 × 1卷积的主要计算发⽣在通道维上。
图5.5展⽰了使⽤输⼊通道数为3、输出通道数为2的1×1卷积核的互相关计算。值得注意的是，输
⼊和输出具有相同的⾼和宽。输出中的每个元素来⾃输⼊中在⾼和宽上相同位置的元素在不同通
道之间的按权重累加。假设我们将通道维当作特征维，将⾼和宽维度上的元素当成数据样本，那
么1 × 1卷积层的作⽤与全连接层等价。

![使用输入通道数为3、输出通道数为2的$1\\times 1$卷积核的互相关计算。输入和输出具有相同的高和宽](../img/conv_1x1.svg)

下⾯我们使⽤全连接层中的矩阵乘法来实现1 × 1卷积。这⾥需要在矩阵乘法运算前后对数据形
状做⼀些调整。

In [13]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    Y = nd.dot(K, X) # 全连接层的矩阵乘法
    return Y.reshape((c_o, h, w))

经验证，做1 × 1卷积时，以上函数与之前实现的互相关运算函数corr2d_multi_in_out等价。

In [18]:
X = nd.random.uniform(shape=(3, 3, 3))
K = nd.random.uniform(shape=(2, 3, 1, 1))
Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)

(Y1 - Y2).norm().asscalar()<1e-6

True

在之后的模型⾥我们将会看到1 × 1卷积层被当作保持⾼和宽维度形状不变的全连接层使⽤。于
是，我们可以通过调整⽹络层之间的通道数来控制模型复杂度
## 小结
- 使⽤多通道可以拓展卷积层的模型参数。
- 假设将通道维当作特征维，将⾼和宽维度上的元素当成数据样本，那么1 × 1卷积层的作⽤与全连接层等价。
- 1 × 1卷积层通常⽤来调整⽹络层之间的通道数，并控制模型复杂度。